# **Cocoon: Clean and Stage your Tables from your Databases**


## **You Need...**

1. LLM API
2. Data Warehouse Con (Snowflake/DuckDB/csv)



In [ ]:
! pip install cocoon_data==0.1.44

In [ ]:
from cocoon_data import *

In [ ]:
#@title  Download example table (skip this step if you have your own table)
import requests

files = {
    "orders.csv": "https://raw.githubusercontent.com/Cocoon-Data-Transformation/cocoon/main/files/orders.csv",
}

# Loop through the files dictionary
for file_name, url in files.items():
    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open file in binary write mode
        with open(file_name, "wb") as file:
            file.write(response.content)
        print(f"{file_name} downloaded successfully.")
    else:
        print(f"Failed to download {file_name}.")

orders.csv downloaded successfully.


In [ ]:
#@title Provide your LLM API

# if you use Open AI
openai.api_key  = ''

# # if you use anthropic
# os.environ["ANTHROPIC_API_KEY"] = ""
# openai.api_type ='claude'

# # if you use Vertex AI, please ensure Claude 3 Opus is enabled
# openai.api_type = 'AnthropicVertex'
# os.environ['AnthropicVertex_region'] = "us-east5"
# os.environ['AnthropicVertex_project_id'] = ""

# test if LLM works
test_message = "hello"
messages = [{"role": "user", "content": test_message}]
response = call_llm_chat(messages, temperature=0.1, top_p=0.1)
print(response['choices'][0]['message']['content'])

Hello! How can I assist you today?


In [ ]:
# @title Provide Data Warehouse Con

# In mem duckdb, that loads the df
con = duckdb.connect(database=':memory:')
df = pd.read_csv("orders.csv")
table_name = "orders"
df.columns = [clean_column_name(col) for col in df.columns]
con.register(table_name, df)

# # Snowflake: specify the con info and table ame
# con = snowflake.connector.connect(
#     # check out your url: {account}.snowflakecomputing.com
#     account="",
#     user="",
#     password="",
#     warehouse="",
#     database="",
#     schema="",
# )
# # Please enter the table to stage/clean
# table_name = "orders"

In [ ]:
query_widget, cocoon_workflow = create_stage_workflow(table_name, con)

In [ ]:
query_widget.display()

In [ ]:
cocoon_workflow.start_workflow()